<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>SVM model via SVR</h2></div>

# Khai báo thư viện cần thiết

In [1]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
# ! pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
# ! pip install spacy
from spacy import load
from urllib.parse import urlparse

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVR
from sklearn.metrics import cohen_kappa_score

/home/dagngyen/miniconda3/envs/min_ds-env/lib/python3.10/site-packages/seaborn/_statistics.py:32: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.1)
  from scipy.stats import gaussian_kde
<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


# Đọc dữ liệu

- Đọc dữ liệu trên Google Drive.

In [2]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

In [3]:
# train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
# train.sample(5)

In [4]:
# test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
# test

- Đọc dữ liệu trên local path.

In [5]:
train = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/train.csv')
train.sample(5)

,essay_id,full_text,score
16606,f4d0ca7,"The idea of reducing car usage is a good idea,...",4
1215,1343a02,Dear fellow citizens I think you should join t...,3
7098,6a5a5ad,The Electoral College is a process that should...,5
12284,b4c0303,The Electoral College is a process by which we...,4
14655,d7d0216,"In ""The Challenge of Exploring Venus"" the auth...",3


In [6]:
test = pd.read_csv('./learning-agency-lab-automated-essay-scoring-2/test.csv')
test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [7]:
# Remove score from train and concate with test
all_data = pd.concat([train.iloc[:,:-1], test], axis=0).reset_index(drop=True)
all_data.sample(5)

,essay_id,full_text
3631,35e2bbc,The Driverless people can cause problems. Many...
4281,3f75229,The Mona Lisa is a very impressive mix of emot...
6543,61d399c,Driveles cars are on the rise and should be al...
17148,fd6e5a7,Driving in and of it self has always had a bad...
9259,88b80dc,The technology that they have produced is very...


# Xử lý dữ liệu

## _Phân tách toàn bộ văn bản thành các đoạn văn_

In [8]:
# # Split full_text into paragraphs by '\n\n' with using regex
# def split_paragraphs(text):
#     new_text = re.split('\n\n', text)
#     return new_text, len(new_text)
# # def split_paragraphs(text):
# #     new_text = text.split('\n\n')
# #     return new_text, len(new_text)

# train['paragraph'], train['paragraph_len'] = zip(*train['full_text'].map(split_paragraphs))
# train

In [9]:
def split_paragraphs(text):
    new_text = re.split('\n\n', text)
    return new_text

train['paragraph'] = train['full_text'].map(split_paragraphs)
train

,essay_id,full_text,score,paragraph
0,000d118,Many people have car where they live. The thin...,3,[Many people have car where they live. The thi...
1,000fe60,I am a scientist at NASA that is discussing th...,3,[I am a scientist at NASA that is discussing t...
2,001ab80,People always wish they had the same technolog...,4,[People always wish they had the same technolo...
3,001bdc0,"We all heard about Venus, the planet without a...",4,"[We all heard about Venus, the planet without ..."
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,"[Dear, State Senator, This is a letter to argu..."
...,...,...,...,...
17302,ffd378d,"the story "" The Challenge of Exploing Venus "" ...",2,"[the story "" The Challenge of Exploing Venus ""..."
17303,ffddf1f,Technology has changed a lot of ways that we l...,4,[Technology has changed a lot of ways that we ...
17304,fff016d,If you don't like sitting around all day than ...,2,[If you don't like sitting around all day than...
17305,fffb49b,"In ""The Challenge of Exporing Venus,"" the auth...",1,"[In ""The Challenge of Exporing Venus,"" the aut..."


## _Loại bỏ các ký tự dư thừa và chuỗi không cung cấp nhiều ý nghĩa thông tin_

In [10]:
def remove_excess_str(text):
    # Loại bỏ các username bắt đầu @
    text = re.sub("@\w+", '', text)
    # Loại bỏ các thẻ HTML
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)
    # Loại bỏ URL
    text = [word for word in text.split() if not urlparse(word).scheme]
    text = ' '.join(text)
    # Loại bỏ dấu nháy đơn theo sau số
    text = re.sub("'\d+", "", text)
    # Loại bỏ các ký tự dư thừa như khoảng trắng, dấu chấm, dấu phẩy
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\.+", ".", text)
    text = re.sub(r"\,+", ",", text)
    text = text.strip()
    text = ' '.join(text.split())
    return text

# Trích lọc đặc trưng từ dữ liệu

Đầu tiên, ta định nghĩa và cài đặt hàm đếm số lượng từ vựng bị sử dụng sai.

In [11]:
nlp = load("en_core_web_sm")

with open('./embedding/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)
    
def count_spelling_errors(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

Tiếp theo, định nghĩa hàm loại bỏ dấu câu để thống kê thông số về từ.

In [12]:
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

Phần này, ta sẽ quan sát những giá trị thống kê từ đoạn văn.
- Đối với 

In [17]:
paragraph = all_data['full_text'].map(split_paragraphs)
paras = paragraph.iloc[0]

def test_engi(paras):
    num_errors = 0
    lst_sentences = []
    lst_words = []
    for para in paras:
        para = remove_excess_str(para)
        num_errors += count_spelling_errors(para)
        # Count number of sentences and words, then append to list
        lst_sentences.append(len(sent_tokenize(para)))
        lst_words.append(len(word_tokenize(para)))
    return num_errors,\
            np.min(lst_sentences), np.quantile(lst_sentences, 0.25), np.median(lst_sentences), np.quantile(lst_sentences, 0.75), np.max(lst_sentences),\
            np.min(lst_words), np.quantile(lst_words, 0.25), np.median(lst_words), np.quantile(lst_words, 0.75), np.max(lst_words)
paragraph.apply(lambda x: test_engi(x))

0        (59, 13, 13.0, 13.0, 13.0, 13, 545, 545.0, 545...
1        (40, 2, 3.0, 3.0, 5.0, 8, 44, 52.0, 81.0, 85.0...
2        (58, 4, 4.0, 5.5, 7.5, 9, 92, 104.75, 149.5, 1...
3        (65, 2, 2.0, 4.0, 6.0, 7, 25, 69.0, 75.0, 149....
4        (52, 1, 1.25, 3.0, 4.0, 4, 3, 22.0, 87.0, 103....
                               ...                        
17305    (41, 11, 11.0, 11.0, 11.0, 11, 264, 264.0, 264...
17306    (19, 1, 1.0, 1.5, 3.5, 8, 4, 16.0, 31.0, 57.0,...
17307    (59, 13, 13.0, 13.0, 13.0, 13, 545, 545.0, 545...
17308    (40, 2, 3.0, 3.0, 5.0, 8, 44, 52.0, 81.0, 85.0...
17309    (58, 4, 4.0, 5.5, 7.5, 9, 92, 104.75, 149.5, 1...
Name: full_text, Length: 17310, dtype: object

(59, 13, 13.0, 13.0, 13.0, 13, 545, 545.0, 545.0, 545.0, 545)

In [14]:
# def feature_engineering(df):
#     # Paragraph engineering
#     paragraph = df['full_text'].map(split_paragraphs)
#     df['num_paragraph'] = paragraph.map(len)
#     def paragraph_engineering(lst):
#         num_errors = 0
#         lst_sentences = []
#         lst_words = []
#         for para in lst:
#             para = remove_excess_str(para)
#             num_errors += count_spelling_errors(para)
#             # Count number of sentences and words, then append to list
#             lst_sentences.append(len(sent_tokenize(para)))
#             lst_words.append(len(word_tokenize(para)))
#         return num_errors,\
#             np.min(lst_sentences), np.quantile(lst_sentences, 0.25), np.median(lst_sentences), np.quantile(lst_sentences, 0.75), np.max(lst_sentences),\
#             np.min(lst_words), np.quantile(lst_words, 0.25), np.median(lst_words), np.quantile(lst_words, 0.75), np.max(lst_words)
#     df['num_errors'],\
#         df['para_min_sentence'], df['para_q1_sentence'], df['para_median_sentence'], df['para_q3_sentence'], df['para_max_sentence'],\
#             df['para_min_word'], df['para_q1_word'], df['para_median_word'], df['para_q3_word'], df['para_max_word'] = zip(*paragraph.map(paragraph_engineering))
#     return df

In [15]:
feature_engineering(all_data)

NameError: name 'feature_engineering' is not defined

In [ ]:
text = "    Xin  chào Việt Nam!... "

text = [word.strip() for word in text.split()]
text = ' '.join(text)

text

'Xin chào Việt Nam!...'

In [ ]:
text = "    Xin  chào Việt Nam... '33 "
remove_excess_str(text)

error: bad escape \d at position 0

In [ ]:
# Load Spacy model
nlp = load("en_core_web_sm")

# Define stop words
stop_words = set(stopwords.words('english'))

# Preprocess the text
def preprocess(text):
    text = text.lower()
    words = word_tokenize(text)
    print(words)
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return words

In [ ]:
preprocess("I am learning Natural Language Processing. We does not have Vietnam.")

['i', 'am', 'learning', 'natural', 'language', 'processing', '.', 'we', 'does', 'not', 'have', 'vietnam', '.']


['learning', 'natural', 'language', 'processing', 'vietnam']

In [ ]:
print(stop_words)

{"you're", 'each', "weren't", 'weren', 'than', 'after', 'are', 'own', 'she', 'be', 'wouldn', 'the', 'while', 'too', 'do', "shouldn't", 'our', 'which', 'down', 'themselves', 'ain', "needn't", 'because', "couldn't", 'does', "isn't", 'won', 'once', 'as', 'we', 'isn', "don't", 'out', 'now', 'd', "wouldn't", 'how', 'this', 't', 'me', 'by', 'under', 'is', 'such', 'here', 'doesn', 'why', 'only', 'ma', "mightn't", 'her', 'nor', 'between', 'there', 'your', 'no', 'y', 'needn', "you'd", 'so', 'before', 'm', 's', 'been', 'ourselves', "aren't", 'herself', 'couldn', 'did', 'himself', 'those', 'during', 'a', 'through', 'until', 'it', 'just', "should've", "didn't", 'up', 'from', "shan't", 'should', 'into', 'of', "hadn't", 'will', 'them', 'below', 'further', 'its', 'doing', 'his', 'they', 'what', 'both', 'don', 'has', 'being', 'very', 'myself', 'any', "hasn't", "you've", 'yours', 'hers', 'with', 'when', 'my', 've', 'all', 'he', 'didn', 'him', 'if', 'these', 'not', "it's", 'll', 'who', 'have', 'an', 'ot

In [ ]:
import string
def remove_punctuation(text):
    """
    Remove all punctuation from the input text.
    
    Args:
    - text (str): The input text.
    
    Returns:
    - str: The text with punctuation removed.
    """

    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

remove_punctuation("I am learning Natural Language Processing. We does not have Vietnam.")

'I am learning Natural Language Processing We does not have Vietnam'

In [ ]:
nlp = load("en_core_web_sm")

with open('./embedding/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)
    
def count_spelling_errors(text):
    doc = nlp(text)
    
    lemmatized_tokens = [token.lemma_.lower() for token in doc]
    spelling_errors = sum(1 for token in lemmatized_tokens if token not in english_vocab)
    return spelling_errors

count_spelling_errors("I am learning Natural Language Processing. We does not have Vietnam.")

2